# importing necessary libraries

In [1]:
import hashlib
import time

# defining the block class

In [2]:
class Block:
    def __init__(self, index, timestamp, vote_info, proof, previous_hash):
         # initializing a block with index, timestamp, vote information, proof of work, & previous hash
        self.index = index
        self.timestamp = timestamp
        self.vote_info = vote_info
        self.proof = proof
        self.previous_hash = previous_hash
        self.hash = self.compute_hash() # computing the hash of the block

    def compute_hash(self):
        # converting the block attributes to a JSON string and compute its SHA-256 hash
        block_string = f"{self.index}{self.timestamp}{self.vote_info}{self.proof}{self.previous_hash}"
        return hashlib.sha256(block_string.encode()).hexdigest()


# defining blockchain class & vote class

In [3]:
class Blockchain:
    def __init__(self):
        # initializing the blockchain with an empty chain and creating the genesis block
        self.chain = []
        self.create_genesis_block()
        # to store votes that are yet to be added to a block
        self.unconfirmed_votes = []  

    def create_genesis_block(self):
        # create and add the genesis block (first block) to the chain
        genesis_block = Block(index=0, timestamp=time.time(), vote_info="Genesis Block", proof=0, previous_hash="0")
        self.chain.append(genesis_block)

    def get_latest_block(self):
        # get the latest block in the chain
        return self.chain[-1]

    def add_block(self, new_block):
        # set the previous hash of the new block to the hash of the latest block
        previous_hash = self.get_latest_block().hash
        new_block.previous_hash = previous_hash
        # compute the hash of the new block
        new_block.hash = new_block.compute_hash()
        # add the new block to the chain
        self.chain.append(new_block)

    def validate_chain(self):
        # validate the entire blockchain
        for i in range(1, len(self.chain)):
            current = self.chain[i]
            previous = self.chain[i - 1]
            # check if the current block's hash is correct
            if current.hash != current.compute_hash():
                return False
            # check if the current block's previous_hash matches the previous block's hash
            if current.previous_hash != previous.hash:
                return False
        return True

    def update_chain(self):
        # placeholder function for updating the blockchain in a network (simplified)
        pass

    def proof_of_work(self, previous_proof):
        # generate a valid proof of work for the given previous proof
        new_proof = 1
        check_proof = False
        while not check_proof:
            hash_operation = hashlib.sha256(str(new_proof**2 - previous_proof**2).encode()).hexdigest()
            if hash_operation[:4] == '0000':  # adjusting the difficulty as needed
                check_proof = True
            else:
                new_proof += 1
        return new_proof

    def create_block(self, proof, previous_hash):
        # create a new block with the given proof and previous hash
        new_block = Block(index=len(self.chain),
                          timestamp=time.time(),
                          vote_info=self.unconfirmed_votes,
                          proof=proof,
                          previous_hash=previous_hash)
        # clear the list of unconfirmed votes
        self.unconfirmed_votes = []
        # add the new block to the chain
        self.add_block(new_block)
        return new_block

    def create_vote(self, voter_name, unique_ID, voted_for):
        # create a new vote and add it to the list of unconfirmed votes
        vote = {'voter_name': voter_name, 'unique_ID': unique_ID, 'voted_for': voted_for}
        self.unconfirmed_votes.append(vote)

    def hash(self, block):
        # compute the hash of the given block
        return block.compute_hash()

    def last_block(self):
        # get the latest block in the chain
        return self.get_latest_block()

# putting it all together

In [4]:
# initializing the blockchain
blockchain = Blockchain()

# adding some votes
blockchain.create_vote("Aaron", "1204", "CandidateA")
blockchain.create_vote("Keira", "5177", "CandidateB")

# create a new block with the added votes
# get the proof of work from the last block
last_proof = blockchain.last_block().proof
# generate a new proof of work
proof = blockchain.proof_of_work(last_proof)
# get the hash of the last block
previous_hash = blockchain.last_block().hash
# create a new block with the proof and previous hash
new_block = blockchain.create_block(proof, previous_hash)

# validate the blockchain
is_valid = blockchain.validate_chain()
print(f"Blockchain valid: {is_valid}")

# output the blockchain
for block in blockchain.chain:
    # print details of each block in the chain
    print(f"Block {block.index}: {block.__dict__}")


Blockchain valid: True
Block 0: {'index': 0, 'timestamp': 1722880656.3307962, 'vote_info': 'Genesis Block', 'proof': 0, 'previous_hash': '0', 'hash': '6a0f34ece479a23d53c12c5530726bf76a1c8449a95dbcf1a8bb5d3a1591c3f8'}
Block 1: {'index': 1, 'timestamp': 1722880656.4345937, 'vote_info': [{'voter_name': 'Aaron', 'unique_ID': '1204', 'voted_for': 'CandidateA'}, {'voter_name': 'Keira', 'unique_ID': '5177', 'voted_for': 'CandidateB'}], 'proof': 115558, 'previous_hash': '6a0f34ece479a23d53c12c5530726bf76a1c8449a95dbcf1a8bb5d3a1591c3f8', 'hash': '2ec43599acdfa0d8b0e39d07af3a86a5d672d6251478bdcedac0fc220546b76c'}
